In [ ]:
import numpy as np
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    Cylinder,
    DiagramBuilder,
    PiecewisePolynomial,
    InverseKinematics,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    RigidTransform,
    Role,
    RollPitchYaw,
    RotationMatrix,
    Solve,
    TrajectorySource,
    StartMeshcat,
    Simulator,
)

from pydrake.multibody import inverse_kinematics
from manipulation import running_as_notebook
from manipulation.meshcat_utils import MeshcatPoseSliders, AddMeshcatTriad
from manipulation.scenarios import AddIiwa, AddShape, AddWsg
from manipulation.station import MakeHardwareStation

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://f7bf94ae-48b6-4da0-b037-4d9acdec7e95.deepnoteproject.com/7001/


In [ ]:
# def teleop_inverse_kinematics():
#     builder = DiagramBuilder()

#     plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
#     iiwa = AddIiwa(plant)
#     wsg = AddWsg(plant, iiwa, welded=True)
#     plant.Finalize()

#     visualizer = MeshcatVisualizer.AddToBuilder(
#         builder,
#         scene_graph,
#         meshcat,
#         MeshcatVisualizerParams(delete_prefix_initialization_event=False),
#     )

#     diagram = builder.Build()
#     context = diagram.CreateDefaultContext()
#     plant_context = plant.GetMyContextFromRoot(context)

#     q0 = plant.GetPositions(plant_context)
#     gripper_frame = plant.GetFrameByName("body", wsg)

#     def my_callback(context, pose):
#         print(type(pose), pose)
#         ik = InverseKinematics(plant, plant_context)
#         ik.AddPositionConstraint(
#             gripper_frame,
#             [0, 0, 0],
#             plant.world_frame(),
#             pose.translation(),
#             pose.translation(),
#         )
#         ik.AddOrientationConstraint(
#             gripper_frame,
#             RotationMatrix(),
#             plant.world_frame(),
#             pose.rotation(),
#             0.0,
#         )
#         prog = ik.get_mutable_prog()
#         q = ik.q()
#         prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
#         prog.SetInitialGuess(q, q0)
#         result = Solve(ik.prog())
#         if result.is_success():
#             print("IK success")
#         else:
#             print("IK failure")
#         clear_output(wait=True)

#     meshcat.DeleteAddedControls()
#     sliders = MeshcatPoseSliders(meshcat)
#     sliders.SetPose(
#         plant.EvalBodyPoseInWorld(
#             plant_context, plant.GetBodyByName("body", wsg)
#         )
#     )
#     sliders.Run(visualizer, context, my_callback)


# teleop_inverse_kinematics()

In [ ]:
# def build_and_simulate(q_traj, duration=0.01):
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
iiwa = AddIiwa(plant)
wsg = AddWsg(plant, iiwa, welded=True)
plant.Finalize()

visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(delete_prefix_initialization_event=False),
)

# meshcat.DeleteAddedControls()
# sliders = MeshcatPoseSliders(meshcat)
# sliders.SetPose(
#     plant.EvalBodyPoseInWorld(
#         plant_context, plant.GetBodyByName("body", wsg)
#     )
# )
# sliders.Run(visualizer, context)]

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/fe5326c5ffc36fda12c58883d22d29dc86009d65.tar.gz


In [ ]:
def move_to_pose(pose):
    print(type(pose), pose)
    ik = InverseKinematics(plant, plant_context)
    # ik = inverse_kinematics.InverseKinematics(plant)
    q_variables = ik.q()  # Get variables for MathematicalProgram
    
    gripper_frame = plant.GetFrameByName("body")


    ik.AddPositionConstraint(
        gripper_frame,
        [0, 0, 0],
        plant.world_frame(),
        pose.translation(),
        pose.translation(),
    )
    ik.AddOrientationConstraint(
        gripper_frame,
        RotationMatrix(),
        plant.world_frame(),
        pose.rotation(),
        0.0,
    )
    prog = ik.get_mutable_prog()
    q = ik.q()
    prog.AddQuadraticErrorCost(np.identity(len(q)), q0, q)
    prog.SetInitialGuess(q, q0)
    result = Solve(ik.prog())
    if result.is_success():
        print("IK success")
    else:
        print("IK failure")
    clear_output(wait=True)


    qf = result.GetSolution()
    print(pose, "-->", result.GetSolution(q_variables))

    return result.GetSolution(q_variables)

sample_pose_vector = np.array([0.34, 0.0, 0.68])
sample_rotation_vector = RotationMatrix.MakeXRotation(0.0).MakeYRotation(0.0).MakeZRotation(0.0)
sample_pose = RigidTransform(sample_rotation_vector, sample_pose_vector)

num_traj_points = 9
qf = move_to_pose(sample_pose)
qfs = []
for i in range(num_traj_points):
    qfs.append(qf)

t_lst = np.linspace(0, 11, num_traj_points)
pose_lst = []
# for t in t_lst:
#     AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePose(t), opacity=0.2)
#     pose_lst.append(InterpolatePose(t))

<class 'pydrake.math.RigidTransform'> RigidTransform(
  R=RotationMatrix([
    [1.0, -0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.34, 0.0, 0.68],
)


NameError: name 'plant_context' is not defined

In [ ]:
# build_and_simulate(q_traj, 11.0)
q_knots = np.array(qfs)
print(qfs, q_knots.shape)
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)
q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
# g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

builder.Connect(
    q_traj_system.get_output_port(), station.GetInputPort("iiwa_position")
)
# builder.Connect(
#     g_traj_system.get_output_port(), station.GetInputPort("wsg_position")
# )

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

q0 = plant.GetPositions(plant_context)
gripper_frame = plant.GetFrameByName("body", wsg)

simulator = Simulator(diagram)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(duration)

[array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113]), array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113]), array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113]), array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113]), array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113]), array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113]), array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113]), array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113]), array([-0.1957127 , -0.03801395, -0.26055104, -1.75756093,  1.48796884,
        2.01991698,  1.40410113])] (9, 7)


RuntimeError: DiagramBuilder: Build() or BuildInto() has already been called to create a Diagram; this DiagramBuilder may no longer be used.

In [ ]:
# simulator.AdvanceTo(10.0)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f7bf94ae-48b6-4da0-b037-4d9acdec7e95' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>